# 実験の再現性をモニタリングする

Snakefileを実行し、実験中に得た結果と同じ結果が得られるかどうかを確認します。  
＊この機能は実験後、**Snakefile記載済**かつ**GIN-forkに同期済**の場合に実行できます。

## 1. 再現性モニタリングのための準備を行う

### 1.1  ユーザー認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってGIN-forkに登録したユーザー名、パスワード、メールアドレスを入力してください。

In [ ]:
%cd ~/WORKFLOWS/EX-WORKFLOWS/util
import json
from scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    email = input("メールアドレス：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    tokens = response.json()

    # 既存のトークンがなければ作成する
    if len(tokens) < 1:
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))

    if response.status_code == HTTPStatus.OK or HTTPStatus.CREATED:
        tokens = response.json()
        clear_output()
        print("認証が正常に完了しました。次の手順へお進みください。")
        break
    else:
        clear_output()
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")

!git config --global user.name $name
!git config --global user.email $email

### 1.2 データ同期のための設定をする

この手順では、今の実行環境とGIN-forkのリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。  

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json
from scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

# コンテナを消す際にコンテナとつなぐための公開鍵も削除のため、
# パラメータとしてGINから発行された鍵IDを保存
if response.status_code == HTTPStatus.CREATED:
    # params.jsonへの追記（鍵ID）
    params['ginKeyId'] = str(response.json()['id'])
    with open(utils.fetch_param_file_path(), mode='w') as f:
        json.dump(params, f, indent=4)
    print('Public key is ready.')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('Public key is ready before time.')

In [ ]:
from datalad import api
from IPython.display import clear_output

# sibling url をsshに変更する
%cd ~/
http_url = !git config --get remote.origin.url
for item in http_url:
    http_url = item
    ssh_url = item.replace(params['siblings']['ginHttp'], params['siblings']['ginSsh'])
    
# siblingsにGINを登録する
sibling = !datalad siblings -s gin
for item in sibling:
    if 'unknown sibling name' in item:
        api.siblings(action='add', name='gin', url=ssh_url)
    else:
        pass

clear_output()
print('SSH connection is ready.')

In [ ]:
# SSHホスト（＝GIN）を信頼する設定
# ドメイン名がハードコーディングにつき要修正
with open('/home/jovyan/.ssh/config', mode='w') as f:
    f.write('host dg02.dg.rcos.nii.ac.jp\n\tStrictHostKeyChecking no\n\tUserKnownHostsFile=/dev/null\n')

## 2. 再現性を確認したい実験パッケージを選択する

以下のセルを実行して、どの実験パッケージの再現性を確認するか選択してください。  
選択を間違えた場合は、再度このセルから実行して、選択しなおしてください。

In [ ]:
import json
from ipywidgets import Dropdown, Button
from IPython.display import clear_output

# 実験パッケージ名を取得
with open('/home/jovyan/pipeline.json', 'r') as f:
    pipeline = json.load(f)

# 再現する実験パッケージを選択
def on_click_callback(clicked_button: Button) -> None:
    global package
    package=dropdown.value
    clear_output()
    print("入力を受けつけました：", package)

dropdown = Dropdown(
    options=pipeline,
    description='実験パッケージ名:',
    disabled=False,
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
display(dropdown, button)

## 3. 実験パッケージを再現するための準備を行う

実験パッケージを再実行するための準備として以下を行います。
- 環境構成ファイルを使用して実行環境を用意する。
- 再実行時の出力データで実験結果が上書きされないように実験時の出力フォルダを移動する。

In [ ]:
import glob
import os

# activate状態のconda環境を特定する
env_list = !conda env list
for env in env_list:
    if '*' in env:
        env = env.split(' ')
        activate_env = env[0]

# 選択された実験パッケージの環境構成ファイルでactivate_envを更新する
os.chdir('/home/jovyan')
package_path = glob.glob('**/'+package, recursive=True)[0]

if os.path.exists('/home/jovyan/' + package_path + '/environment.yml'):
    !conda env update --name $activate_env --file $package_path/environment.yml
if os.path.exists('/home/jovyan/' + package_path + '/requirements.txt'):
    !pip install -r $package_path/requirements.txt
    
# 出力データが上書きされないように、output_data配下をformer_output_data配下に移動する
%cd ~/
!datalad get $package_path
%cd ~/
!mkdir -p $package_path/former_output_data
!mv $package_path/output_data/* $package_path/former_output_data/

## 4. 実験パッケージを再実行する

実験パッケージの実験を再実行します。  
以下でエラーが起きた場合は、Snakefileと実験ソースコードが正しく記述されているかを確認してください。  
一度、終了した実験を編集・実行する方法については[こちら](https://github.com/NII-DG/dg-manual#%E5%AE%9F%E9%A8%93%E3%81%AE%E6%B5%81%E3%82%8C)をご覧ください。

In [ ]:
%cd ~/
!snakemake --snakefile $package_path/Snakefile --cores all

## 5. 実験パッケージのoutput_dataと再現時の実験結果が一致するかを比較する

実験パッケージのoutput_dataと再現時の実験結果が一致しているかどうかを表示します。  
表示された結果に関わらず、最後までこのワークフローを実行してGIN-forkに再現結果を反映してください。  
「実験結果が一致しません。」と表示された場合は、このワークフローの実行完了後、[こちら](https://github.com/NII-DG/dg-manual#%E5%AE%9F%E9%A8%93%E3%81%AE%E6%B5%81%E3%82%8C)の手順に従って実験パッケージの編集や再実行を行ってください。

In [ ]:
# 年月日を除いたファイル名が同じである比較対象のファイルをリストのまとめる
import glob

former_output_list = glob.glob(package_path + '/former_output_data/**', recursive=True)
for path in former_output_list:
    if os.path.isdir(path):
        former_output_list.remove(path)

output_list = glob.glob(package_path + '/output_data/**', recursive=True)
for path in output_list:
    if os.path.isdir(path):
        output_list.remove(path)

comparison_list = []
for former_path in former_output_list:
    for output_path in output_list:
        if former_path[:8] == output_path[:8]:
            list = [former_path, output_path]
            comparison_list.append(list)

In [ ]:
# ファイルを比較する
import os

os.chdir('/home/jovyan')
isSucceeded = True
for list in comparison_list:
    cmd = 'diff ' + list[0] + ' ' + list[1]
    result = !$cmd
    if len(result) > 0:
        isSucceeded = False
        print("実験結果が一致しません。：", list)
    else:
        print("出力データが一致しました。：", list)

In [ ]:
import os

# モニタリング結果を実験パッケージのREADMEに反映する
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils 
utils.reflect_monitoring_results('reproducibility', isSucceeded, '/home/jovyan/' + package_path)

# 再度、実行される場合に備えて実験時の出力データを元に戻す
!rm -rf ~/$package_path/output_data
!mv ~/$package_path/former_output_data ~/$package_path/output_data

## 6. 実験パッケージの再現結果をGIN-forkに反映する

実験パッケージの再現性の確認結果をGIN-forkに反映します。  
GIN-forkの各実験パッケージのREADME.mdから再現性モニタリングの結果をご確認いただけます。  

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

# 実験記録を上書きしない為に、このノートブックと実験パッケージのREADMEのみを書き戻す
os.chdir('/home/jovyan')
README_path = '/home/jovyan/' + package_path + '/README.md'
save_path = [README_path, '/home/jovyan/WORKFLOWS/monitor_package.ipynb']

try:
    pm.execute_notebook(
        'WORKFLOWS/FLOW/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = 'モニタリング（再現性）', IS_RECURSIVE = False, TO_GIT = True, PATH = save_path)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 7. この実行環境を削除する

この実行環境を停止・削除します。  

### 7.1 この実行環境を確認する  
以下のセルを実行して実行環境のサーバー名を確認して下さい。

In [ ]:
import os
print(os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3])

### 7.2 コントロールパネルへ遷移し、実行環境を停止・削除する

[コントロールパネル](https://jupyter.cs.rcos.nii.ac.jp/hub/home)へ遷移して、`7.1`で確認したサーバーを`stop`、`delete`ボタンをクリックして停止・削除してください。  
※`delete`ボタンは、以下の図のように`stop`ボタンをクリックした後に表示されます。  
![コンテナ削除キャプチャ](./EX-WORKFLOWS/images/コンテナ削除キャプチャ.png)